# Connectivity Cost Estimation

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import json
import os
import pickle
import binascii
from ipywidgets import FileUpload, Output, Label, HBox, HTML, Layout

from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput
from giga.viz.notebooks.components.widgets.giga_file_upload import GigaFileUpload
from giga.viz.notebooks.components.html.sections import section

import warnings
warnings.filterwarnings('ignore')

show_map = os.environ.get("FF_STATIC_MAP_NOTEBOOK", True)

output = Output()

upload = GigaFileUpload(accept='.json', multiple=True, description="Import Config")
inputs = CostEstimationParameterInput(local_data_workspace='workspace', show_map=show_map)

upload_package = GigaFileUpload(accept='.pkl', multiple=False, description="Results Package (.pkl)")
pkg = None

def on_upload_change(change):
    try:
        imported = json.loads(upload.value[0].content.tobytes())
    except IndexError:
        imported = {}
    output.clear_output()
    inputs.update(imported)
    
def on_upload_package_change(change):
    global pkg
    if len(upload_package.value) != 1:
        pkg = None
        return
    try:
        decoded_data = binascii.unhexlify(upload_package.value[0].content.tobytes().decode('utf-8'))
        pkg = pickle.loads(decoded_data)
        new_inputs = pkg.config
        sel_schools = pkg.selected_schools
    except:
        new_inputs = {}
        sel_schools = []
        pkg = None
    output.clear_output()
    inputs.update(new_inputs)
    inputs.set_selected_schools(sel_schools)
    inputs.freeze()

upload.observe(on_upload_change, names='value')
upload_package.observe(on_upload_package_change, names='value')

display(section(
    title="Configuration",
    contents=HBox([
        upload,
        HTML("For help, see documentation on <a href=\"docs/main.ipynb\" style=\"text-decoration:underline\">configuring and running these models</a>."),
    ]),
    extra_class="header"
))

display(section(
    title="Results drilldown",
    contents=HBox([
        upload_package,
        HTML("For help, see documentation on <a href=\"docs/main.ipynb\" style=\"text-decoration:underline\">configuring and running these models</a>."),
    ]),
    extra_class="header"
))

display(inputs.parameter_input(), output)

In [ ]:
from ipywidgets import Button, Output, HBox, VBox, HTML, GridBox
from IPython.display import clear_output, Javascript
import json

from giga.models.scenarios.scenario_dispatcher import create_scenario
from giga.data.space.model_data_space import ModelDataSpace
from giga.schemas.output import OutputSpace
from giga.data.stats.result_stats import ResultStats
from giga.viz.notebooks.components.dashboard.result_dashboard import ResultDashboard
from giga.utils.logging import LOGGER
from giga.viz.notebooks.tables import display_summary_table, plot_cost_breakdown, output_to_capex_details
from giga.viz.notebooks.helpers import output_to_table
from giga.viz.notebooks.components.html.sections import section
from giga.viz.notebooks.components.widgets.giga_buttons import make_run_button, make_run_again_button, make_show_maps_button, make_show_full_table_button
from giga.viz.notebooks.components.widgets.giga_export import make_export_button_row, make_export_infra_report_button

result_output = Output()

output_space = None
data_space = None
config = None
button = None
results_table = None
data_space_lookup = {}
data_space_selected = None
dashboard = None
map_output = Output()
table_output = Output()

def show_all_maps(event):
    dashboard.display_result_maps_output(map_output)

#def show_full_table(event):
#    with table_output:
#        table = HTML(results_table.to_html(col_space=5, border=0))
#        display(section("Cost Table", table))

def on_run_button_clicked(b):
    global config
    global data_space
    global output_space
    global results_table
    global button_infra
    global upload_package
    global dashboard
    global data_space_selected
    global map_output
        
    del(results_table)
    del(dashboard)
    del(output_space)
    
    # freeze and disable confgiuration + running while model in progress
    b.disabled = True
    button_infra.disabled = True
        
    with result_output:
        if len(inputs.get_selected_schools()) == 0:
            display(HTML("<b>No Schools Selected, Please Select Schools Before Running Models<b>"))
            # freeze and disable confgiuration + running while model in progress
            b.disabled = False
            button_infra.disabled = False
            return
        clear_output(wait=True)
        
        # pull verbose flag
        verbose = inputs.dashboard_parameters()["verbose"]
        LOGGER.propagate = verbose
        if pkg!=None:
            LOGGER.info(f"Loading results package data")
            # Get selected schools
            school_ids = inputs.get_selected_schools()
            
            # Reconstruct a data space
            data_space = ModelDataSpace(inputs.data_parameters())
            data_space_selected = data_space.filter_schools(school_ids)
        
            # Reconstruct an output space
            output_space = pkg.output_space
            output_space_selected = output_space.filter_schools(school_ids)
            
            # Construct a results dashboard
            stats = ResultStats(data_space_selected, output_space_selected, inputs.all_tech_config())
            dashboard = ResultDashboard(stats, inputs)
            
            n_opex_years = inputs.scenario_parameters().years_opex
            results_table = data_space_selected.school_outputs_to_frame(output_space_selected.full_results_table(n_opex_years))
            
            export_buttons = make_export_button_row(dashboard, results_table)
            
            #empty pkg
            upload_package.value = ()
        else:
            inputs.freeze()
        
            # setup scenario
            config = inputs.scenario_parameters()
            country = inputs.data_parameters().school_data_conf.country_id
            if country not in data_space_lookup:
                data_space_lookup[country] = ModelDataSpace(inputs.data_parameters())
            
            # Construct data space
            data_space = data_space_lookup[country]
            data_space_selected = data_space.filter_schools(inputs.get_selected_schools())
            
            output_space = OutputSpace()
            scenario = create_scenario(config, data_space_selected, output_space)
            
            # run the models
            output_space = scenario.run(progress_bar=verbose)
            
            n_opex_years = inputs.scenario_parameters().years_opex
            results_table = data_space_selected.school_outputs_to_frame(output_space.full_results_table(n_opex_years))
            
            # Construct a results dashboard
            stats = ResultStats(data_space, output_space, inputs.all_tech_config())
            dashboard = ResultDashboard(stats, inputs)
            export_buttons = make_export_button_row(dashboard, results_table)
            
        # populate all output
        dashboard.populate_outputs()
        display(HTML('<br/><br/>'))    
        dashboard.display()

        run_again_button = make_run_again_button(inputs, [result_output, map_output, table_output], b, button_infra)
        
        #show_maps_button = make_show_maps_button(show_all_maps)
        #show_maps_button.on_click(show_all_maps)

        #full_table_button = make_show_full_table_button(show_full_table)
        #full_table_button.on_click(show_full_table)

        # Create a grid with two columns, splitting space equally
        layout = Layout(grid_template_columns='1fr 1fr')
        display(GridBox([
            section("More", VBox([
                run_again_button,
                HTML("<hr/>"),
                #show_maps_button,
                #full_table_button
            ])),
            section("Export", export_buttons)
        ], layout=layout))

        js_code = """
        window.dispatchEvent(new Event('resize'));
        """
        display(Javascript(js_code))
        
        
button = make_run_button(on_run_button_clicked)
button.on_click(on_run_button_clicked)

button_infra = make_export_infra_report_button(inputs = inputs, title="Generate Infrastructure Report", filename="infra_report.zip")
#button_infra.on_click(on_infra_button_clicked)
#button_out_infra = VBox([button_infra,infra_output])
display(section(
    title="Run the Model",
    contents=VBox([
        HBox([button,button_infra]),
        HTML("<br/>"),
        result_output,
        map_output,
        table_output
    ]),
    extra_class="run"
))